In [1]:
from models import build_model_bert4Rec
from dataloader import Bert4RecDataLoader

import polars as pl
from tqdm import tqdm
tqdm.pandas()
import gc
from collections import defaultdict
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from datetime import datetime

physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print('Invalid device or cannot modify virtual devices once initialized.')

2022-11-29 16:56:11.756704: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 16:56:11.821188: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-29 16:56:11.836481: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-29 16:56:12.174888: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-11-29 16:56:12.590043: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-29 16:56:12.603572: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-29 16:56:12.603664: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
df_mapping = pd.read_csv('../tfrecords/tfrecords_v0.5/df_mapping.csv')
NUM_ITEMS = len(df_mapping['aid_map'].unique())

dict_map = {}
for x in tqdm(df_mapping.to_dict('records')):
    dict_map[x['aid_map']] = x['aid']

dict_map_type = {
    'clicks' : 1,
    'carts' : 2,
    'orders' : 3
  }

print(f'NUM_ITEMS: {NUM_ITEMS}')

100%|██████████| 1855603/1855603 [00:00<00:00, 8005623.49it/s]

NUM_ITEMS: 1311743


In [3]:
list_paths = ['../tfrecords/tfrecords_v0.5/na_split=val/' + x for x in os.listdir('../tfrecords/tfrecords_v0.5/na_split=val')]
# 5,45, 1,09
dataloader = Bert4RecDataLoader(list_paths, 
                                num_items=NUM_ITEMS, 
                                seq_len=80, 
                                seq_len_target=None,
                                batch_size=32, 
                                mask_prob=0.0, 
                                reverse_prob=0.0, 
                                get_session=True,
                                is_val=True,
                                is_test=False,
                                shuffle=False).get_generator()
# Val
list_sessions_val, list_items_val, list_types_val = [], [], []
for batch in tqdm(dataloader):
    features, targets, session = batch
    seq_items, seq_type, seq_time, seq_recency = features
    list_sessions_val = list_sessions_val + session.numpy().tolist()
    list_items_val += [[dict_map[item]-1 for item in seq if item!= 0] for seq in seq_items[:, :, 0].numpy()]
    list_types_val += [[type_ for type_ in seq if type_!= 0] for seq in seq_type[:, :, 0].numpy()]
    # break
print(len(list_sessions_val))
# 20110it [00:27, 720.48it/s]

# del dataloader, batch, features, targets, session
# gc.collect()

2022-11-29 16:56:14.190954: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 16:56:14.191912: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-29 16:56:14.192057: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-29 16:56:14.192132: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA

643503


In [4]:
# df_train_all = pl.read_parquet(
#     '../0_Data/data_optimized/train.parquet'
# )


# # df_train = df_train_all.filter(~(pl.col('session').is_in(list_sessions_val)))
# df_val = df_train_all.filter(pl.col('session').is_in(list_sessions_val))

# 1. Validation

## 1.1 Rule based predictions 

In [5]:
session_types = ['clicks', 'carts', 'orders']

# val_session_items = df_val.to_pandas().reset_index(drop=True).groupby('session')['aid'].apply(list)
# val_session_types = df_val.to_pandas().reset_index(drop=True).groupby('session')['type'].apply(list)
# val_sessions = val_session_items.index

num_items_to_consider = 20
type_weight_multipliers = {1: 1, 2: 6, 3: 3}
list_labels, list_sessions = [], []
for session, seq_items, seq_types in tqdm(zip(list_sessions_val, list_items_val, list_types_val), total=len(list_items_val)):
    if len(seq_items) >= num_items_to_consider:
        # if we have enough aids (over equals num_items_to_consider) we don't need to look for candidates! we just use the old logic
        weights=np.logspace(0.1, 1, len(seq_items), base=2, endpoint=True)-1
        aids_temp=defaultdict(lambda: 0)
        for aid,w,t in zip(seq_items, weights, seq_types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
            
        sorted_aids=[k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
        list_labels.append(sorted_aids[:num_items_to_consider])
    else:
        list_labels.append([0])
    list_sessions.append(session)

# del list_items_val, list_types_val
# gc.collect()

df_val_rule = pd.DataFrame({
    'session' : list_sessions,
    'prediction' : list_labels,
})

100%|██████████| 643503/643503 [00:01<00:00, 431766.31it/s]


## 1.2 Model based predictions

In [6]:
class BERT4REC_CONFIG:
    seed = 42 
    num_items = NUM_ITEMS
    path_tfrecords = '../tfrecords/tfrecords_v0.5/'
    restore_last_chekpoint = (True, 'model_bert4rec_complete_0.10/checkpoints/', 'ckpt-22')
    model_name = 'model_bert4rec_complete_0.10'
    checkpoint_filepath = f'../2_Models/'
    num_records_dataset = 10_000_000
    batch_size = 32
    tup_scheduler_grad_accum = (5, 10, 1_500_000) #(start_grad_accum, max_grad_accum, ramp_up_samples)
    seq_len = 30
    mask_prob = 0.4
    reverse_prob = 0.5
    emb_dim = 128
    trf_dim = 128
    num_heads = 4
    num_layers = 1
    ff_dim = trf_dim*4
    drop_rate = 0.1
    att_drop_rate = 0.1
    epochs = 3
    early_stopping = 5
    batch_num_printer_train = 500
    batch_num_printer_val = 250
    clipnorm = 1.0
    num_iters_save_checkpoint = 25_000
    scheduler_scaler = 128 
    warmup_steps = 10_000
    weight_decay = 1e-1
    log_wandb = True

def get_score_metric(y_true, y_pred, type_target, k=20):
    score = 0 
    if len(y_true)==0:
        return None
    if type_target=='clicks':
        num_targets = 1
        hits = len([x for x in y_pred if x==y_true[0]])
    else:
        num_targets = min(k, len(y_true))
        hits = len([x for x in y_pred if x in y_true])
    score = hits / num_targets
    return score

In [7]:
tf.keras.backend.clear_session()
model = build_model_bert4Rec(num_items=NUM_ITEMS, model_cfg=BERT4REC_CONFIG)
ckpt = tf.train.Checkpoint(model=model)
ckpt.restore(tf.train.latest_checkpoint(f'../2_Models/model_bert4rec_complete_0.10/checkpoints'))
list_paths_val = ['../tfrecords/tfrecords_v0.5/na_split=val/' + x for x in os.listdir('../tfrecords/tfrecords_v0.5/na_split=val')]
val_dataloader = Bert4RecDataLoader(list_paths_val, 
                                     num_items=NUM_ITEMS, 
                                     seq_len=30, 
                                     seq_len_target=30, 
                                     batch_size=32, 
                                     mask_prob=0.0, 
                                     reverse_prob=0.0, 
                                     is_val=True,
                                     get_session=True, 
                                     is_test=False,
                                     shuffle=False).get_generator()


list_sessions, list_past_items, list_predictions, list_trues, list_types = [], [], [], [], []
for num_batch, batch in enumerate(tqdm(val_dataloader)):
    features, targets, session = batch
    seq_items, seq_type, seq_time, seq_recency = features
    target, type_target, idx_mask = targets
    idxs = idx_mask.numpy()
    for type_ in ['clicks', 'carts', 'orders']:
        seq_type_new = [tf.concat([
                        seq_type[i, :ix],
                        tf.constant([[dict_map_type[type_]]], tf.int64),
                        seq_type[i, ix+1:]], axis=0)
                    for i, ix in enumerate(idxs)]
        features = (seq_items, tf.stack(seq_type_new, axis=0), seq_time, seq_recency)
        preds = model(features, training=False)
        preds = tf.gather(preds, indices=idxs, axis=1, batch_dims=1)
        topk_scores, topk_idxs = tf.math.top_k(preds, k=20)
        topk_idxs = np.asarray([[dict_map[x]-1 for x in topk_idxs.numpy()[i, :]] for i in range(topk_idxs.numpy().shape[0])])
        labels = [list(set([dict_map[_target]-1 for _type, _target in zip(type_target.numpy()[i], target.numpy()[i]) if dict_map_type[type_]==_type and _target!=0])) for i in range(target.shape[0])]
        ###
        list_sessions.append(session.numpy())
        list_predictions.append(topk_idxs)
        list_types.append([type_ for _ in range(seq_items.shape[0])])
        list_trues = list_trues + labels
        list_past_items.append(seq_items.numpy()[:, :, 0])
    if num_batch==500:
        break

df_val_model = pd.DataFrame({
    'session' : np.concatenate(list_sessions),
    'past_items' : np.concatenate(list_past_items).tolist(),
    'predictions' : np.concatenate(list_predictions).tolist(),
    'trues' : list_trues,
    'type' : np.concatenate(list_types)
})

df_val_model['qt_trues'] = df_val_model['trues'].apply(lambda x : len(x))

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090 Ti, compute capability 8.6


2022-11-29 16:57:26.168286: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
0it [00:00, ?it/s]2022-11-29 16:57:27.107617: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
500it [00:54,  9.12it/s]


In [8]:
df_val_merge = df_val_rule.merge(df_val_model, how='inner', on='session')
df_val_merge['predictions_final'] = df_val_merge.progress_apply(lambda x : x['predictions'] if len(x['prediction'])==1 else x['prediction'], axis=1)
df_val_merge['score'] = df_val_merge.progress_apply(lambda x: get_score_metric(x['trues'], x['predictions_final'], x['type']), axis=1)

display(df_val_merge.describe())
dict_scores = df_val_merge.groupby('type')['score'].mean().to_dict()
display(dict_scores)
kaggle_metric = 0.1*dict_scores['clicks'] + 0.3*dict_scores['carts'] + 0.6*dict_scores['orders']
print(f'Kaggle Metric: {kaggle_metric:.4f}')

# # Model
# {'carts': 0.36702384612006494,
#  'clicks': 0.3259753593429158,
#  'orders': 0.5007742817594869}
# Kaggle Metric: 0.4432

# rule
# {'carts': 0.062799906313586,
#  'clicks': 0.02098305954825462,
#  'orders': 0.13754458013032322}
# Kaggle Metric: 0.1035

# final
# {'carts': 0.3764704375448304,
#  'clicks': 0.3295687885010267,
#  'orders': 0.5595109534416864}
# Kaggle Metric: 0.4816

100%|██████████| 48096/48096 [00:00<00:00, 222382.83it/s]


,session,qt_trues,score
count,4.809600e+04,48096.000000,20324.000000
mean,6.409855e+06,1.560941,0.352768
std,3.715819e+06,3.557595,0.466305
min,9.630000e+02,0.000000,0.000000
25%,3.190015e+06,0.000000,0.000000
50%,6.383588e+06,0.000000,0.000000
75%,9.627672e+06,1.000000,1.000000
max,1.289967e+07,30.000000,1.000000


{'carts': 0.3770065726874826,
 'clicks': 0.3293762833675565,
 'orders': 0.5448876573624388}

Kaggle Metric: 0.4730


# Test

In [ ]:
del df_val_merge, df_val_model, df_val_rule, list_sessions_val, list_items_val, list_types_val
gc.collect()

## 2.1 Rule based

In [9]:
df_test = pl.read_parquet(
    '../0_Data/data_optimized/test.parquet'
)

In [10]:
session_types = ['clicks', 'carts', 'orders']

test_session_items = df_test.to_pandas().reset_index(drop=True).groupby('session')['aid'].apply(list)
test_session_types = df_test.to_pandas().reset_index(drop=True).groupby('session')['type'].apply(list)
test_sessions = test_session_items.index

num_items_to_consider = 20
type_weight_multipliers = {0: 1, 1: 6, 2: 3}
list_labels, list_sessions = [], []
for session, seq_items, seq_types in tqdm(zip(test_sessions, test_session_items, test_session_types), total=len(test_session_items)):
    if len(seq_items) >= num_items_to_consider:
        # if we have enough aids (over equals 20) we don't need to look for candidates! we just use the old logic
        weights=np.logspace(0.1, 1, len(seq_items), base=2, endpoint=True)-1
        aids_temp=defaultdict(lambda: 0)
        for aid,w,t in zip(seq_items, weights, seq_types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
            
        sorted_aids=[k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
        list_labels.append(sorted_aids[:num_items_to_consider])
    else:
        list_labels.append([0])
    list_sessions.append(session)

# del test_session_items, test_session_types
# gc.collect()

df_test_rule = pd.DataFrame({
    'session' : list_sessions,
    'prediction' : list_labels,
})

100%|██████████| 1671803/1671803 [00:02<00:00, 782559.23it/s] 


## 2.2 Model based predictions

In [11]:
list_paths_test = ['../tfrecords/tfrecords_v0.5/na_split=test/' + x for x in os.listdir('../tfrecords/tfrecords_v0.5/na_split=test')]
test_dataloader = Bert4RecDataLoader(list_paths_test, 
                                     num_items=NUM_ITEMS, 
                                     seq_len=30,  
                                     batch_size=40, 
                                     mask_prob=0.0, 
                                     reverse_prob=0.0,  
                                     is_val=False,
                                     is_test=True,
                                     get_session=True,
                                     shuffle=False).get_generator()

list_predictions, list_sessions, list_types, list_scores = [], [], [], []
for num_batch, batch in enumerate(tqdm(test_dataloader)):
    features, idxs, session = batch
    seq_items, seq_type, seq_time, seq_recency = features
    idxs = idxs.numpy()
    ###
    for type_ in ['clicks', 'carts', 'orders']:
        seq_type_new = [tf.concat([
                        seq_type[i, :ix],
                        tf.constant([[dict_map_type[type_]]], tf.int64),
                        seq_type[i, ix+1:]], axis=0)
                    for i, ix in enumerate(idxs)]
        features = (seq_items, tf.stack(seq_type_new, axis=0), seq_time, seq_recency)
        preds = model(features, training=False)
        preds = tf.gather(preds, indices=idxs, axis=1, batch_dims=1)
        topk_scores, topk_idxs = tf.math.top_k(preds, k=20)
        topk_idxs = np.asarray([[dict_map[x] for x in topk_idxs.numpy()[i, :]] for i in range(topk_idxs.numpy().shape[0])])
        topk_idxs = topk_idxs - 1
        list_predictions.append(topk_idxs)
        list_types.append([type_ for _ in range(seq_items.shape[0])])
        list_sessions.append(session.numpy())

26337it [51:28,  8.23it/s]

In [ ]:
name_submission = f"submission_{datetime.now().__str__().split('.')[0].replace(' ', '_').replace('-', '_').replace(':', '_')}"

df_inference = pd.DataFrame({
    'session' : np.concatenate(list_sessions),
    'predictions' : np.concatenate(list_predictions).tolist(),
    'type' : np.concatenate(list_types)
})

df_inference = df_inference.merge(df_test_rule, how='inner', on='session')
df_inference['predictions_final'] = df_inference.progress_apply(lambda x : x['predictions'] if len(x['prediction'])==1 else x['prediction'], axis=1)

# df_inference['session_type'] = df_inference['session'].astype(str) + '_' + df_inference['type']
# df_inference['labels'] = df_inference['predictions_final'].apply(lambda x : ' '.join([str(y) for y in x]))
# df_inference[['session_type', 'labels']].to_csv(f'../3_Submissions/{name_submission}.csv', index=False)

# print(df_inference.shape)
# display(
#     df_inference
# )

# import gzip
# with open(f'../3_Submissions/{name_submission}.csv', 'rb') as f_in, gzip.open(f'../3_Submissions/{name_submission}.csv.gz', 'wb') as f_out:
#     f_out.writelines(f_in)